In [1]:
import requests as r
import pandas as pd
import numpy as np
import os

import sys
sys.path.append("../../helper_functions")
import duneapi_utils as du
import pandas_utils as p
import google_bq_utils as bqu
sys.path.pop()

import time

In [2]:
table_name = 'op_collective_revenue_transactions'

trailing_days = 90

In [3]:
# Run Dune
print('     dune runs')
days_param = du.generate_query_parameter(input=trailing_days,field_name='trailing_days',dtype='number')
dune_df = du.get_dune_data(query_id = 3046107, #https://dune.com/queries/3046107
    name = table_name,
    path = "outputs",
    performance="medium",
    params = [days_param],
    num_hours_to_rerun=0 #always rereun
)
dune_df['source'] = 'dune'
dune_df['tx_block_time'] = pd.to_datetime(dune_df['tx_block_time']).dt.tz_localize(None)

2024-07-23 14:23:00.017 | INFO     | duneapi_utils:get_dune_data:58 - Results available at https://dune.com/queries/3046107?trailing_days=9990


     dune runs


2024-07-23 14:23:01,282 INFO dune_client.api.base executing 3046107 on medium cluster
2024-07-23 14:23:01,585 INFO dune_client.api.base waiting for query execution 01J3GDA77Y5CN9Z6AP78ED9VRX to complete: ExecutionState.PENDING (queue position: 2)
2024-07-23 14:23:02,728 INFO dune_client.api.base waiting for query execution 01J3GDA77Y5CN9Z6AP78ED9VRX to complete: ExecutionState.PENDING (queue position: 2)
2024-07-23 14:23:03,873 INFO dune_client.api.base waiting for query execution 01J3GDA77Y5CN9Z6AP78ED9VRX to complete: ExecutionState.EXECUTING
2024-07-23 14:23:05,023 INFO dune_client.api.base waiting for query execution 01J3GDA77Y5CN9Z6AP78ED9VRX to complete: ExecutionState.EXECUTING
2024-07-23 14:23:06,178 INFO dune_client.api.base waiting for query execution 01J3GDA77Y5CN9Z6AP78ED9VRX to complete: ExecutionState.EXECUTING
2024-07-23 14:23:07,336 INFO dune_client.api.base waiting for query execution 01J3GDA77Y5CN9Z6AP78ED9VRX to complete: ExecutionState.EXECUTING
2024-07-23 14:23:08,

In [4]:
dune_df['value'] = dune_df['value'].astype(str)

In [5]:
#BQ Upload
bqu.append_and_upsert_df_to_bq_table(dune_df, table_name, unique_keys = ['tx_block_number','tx_hash','trace_address'])

Start Writing api_table_uploads.op_collective_revenue_transactions_staging
Data loaded successfully to api_table_uploads.op_collective_revenue_transactions_staging


2024-07-23 14:24:20,449 INFO google_bq_utils Append and upsert to api_table_uploads.op_collective_revenue_transactions completed successfully
2024-07-23 14:24:20,671 INFO google_bq_utils Staging table oplabs-tools-data.api_table_uploads.op_collective_revenue_transactions_staging deleted.
